# Project Title
## Data Engineering Capstone Project

### Project Summary
* The goal of this project is to evaluate the impact of weather's temperature on immagrants movements over April, 2016 in USA
* Apache Spark is used to extract and transform raw data, and make a datawarehouse in parquet file format. 
* The star schema is used to develop a database, which will be effectively used for handling analytical queries.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

### Step 1: Scope the Project and Gather Data

#### Scope 
-- Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>
* This project extracts raw data from two sources as described below. 
* It creates a datamodel of immagrants' movement in US consistsing of one fact tables referencing two dimension tables.

#### Describe and Gather Data 
-- Describe the data sets you're using. Where did it come from? What type of information is included? 

##### I94 Immigration Data: 
* This data comes from the [US National Tourism and Trade Office](https://travel.trade.gov/research/reports/i94/historical/2016.html).
* Dataset includes infoes on individual incomming immigrants and thei ports on entry. 
* Data dictionary: utils/I94_SAS_Labels_Descriptions.SAS
* Sample file: sample_data/immigration_data_sample.csv
* Columns:
    * 'i94yr': '4 digit year',
    * 'i94mon': 'Numeric month',
    * 'i94addr': 'where the immigrants resides in USA',
    * 'i94port' : 'entry port of immagrant'

##### World Temperature Data:
* This dataset comes from [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).
* Dataset includes infoe on temprature on cities globally. This project only uses data of US cities.
* Columns:
    * 'dt': 'data of recorded temperature'
    * 'AverageTemperature': 'the average temperature recorded over a month',
    * 'City': 'city name',
    * 'Country': 'country name',

In [1]:
# do all imports and installs here
import logging
logging.getLogger().setLevel(logging.INFO)
import pyspark.sql.dataframe as psd
import pyspark.sql.session as pss
from pyspark.sql import SparkSession
from utils.reader import Reader
from utils.help_functions import *

In [2]:
# initialize reader classs
reader = Reader()

INFO:root:geting root paths to read and write data
INFO:root:getting spark session
INFO:root:stage raw data into parquet files if not already exist!!!
INFO:root:i94 immigration raw data are are already written into parquet files
INFO:root:world temperature raw data are are already written into parquet files
INFO:root:Start fitting i94immigration data to reader class ...
INFO:root:End fitting i94immigration data to reader class!
INFO:root:Start fitting worldtemperature data to reader class ...
INFO:root:End fitting worldtemperature data to reader class!


In [3]:
spark = reader.spark
immigration_df = reader.immigration_df
temperature_df = reader.temperature_df

In [4]:
spark

In [5]:
immigration_df.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [6]:
temperature_df.show(5)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

## Data Wrangling: Immigration data

In [7]:
# filter needed columns
immigration_usa_df = filter_immigration_data(immigration_df)

In [8]:
# print schema
immigration_usa_df.printSchema()

root
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- i94port: string (nullable = true)



In [9]:
# print immigration data 
immigration_usa_df.limit(10).toPandas()

,i94yr,i94mon,i94addr,i94port
0,2016.0,4.0,CA,LOS
1,2016.0,4.0,NV,LOS
2,2016.0,4.0,WA,LOS
3,2016.0,4.0,WA,LOS
4,2016.0,4.0,WA,LOS
5,2016.0,4.0,HI,HHW
6,2016.0,4.0,HI,HHW
7,2016.0,4.0,HI,HHW
8,2016.0,4.0,FL,HOU
9,2016.0,4.0,CA,LOS


In [10]:
# create view for immigration data
immigration_usa_df.createOrReplaceTempView('immigration_usa_table')

In [11]:
# count unique values in a given spark sql table
def value_counts(table, col):
    return reader.spark.sql(f"""
    select {col}, count({col}) as count
    from {table}
    group by {col}
    order by count
    """).show(20)

In [12]:
# count unique values of each column
value_counts('immigration_usa_table', 'i94yr')
value_counts('immigration_usa_table', 'i94mon')
value_counts('immigration_usa_table', 'i94addr')
value_counts('immigration_usa_table', 'i94port')

+------+-------+
| i94yr|  count|
+------+-------+
|2016.0|3096313|
+------+-------+

+------+-------+
|i94mon|  count|
+------+-------+
|   4.0|3096313|
+------+-------+

+-------+-----+
|i94addr|count|
+-------+-----+
|   null|    0|
|     KF|    1|
|     52|    1|
|     71|    1|
|     S6|    1|
|     85|    1|
|     UL|    1|
|     RU|    1|
|     VL|    1|
|     RA|    1|
|     UR|    1|
|     ZN|    1|
|     TC|    1|
|     PD|    1|
|     YH|    1|
|     EX|    1|
|     RF|    1|
|     RO|    1|
|     73|    1|
|     FC|    1|
+-------+-----+
only showing top 20 rows

+-------+-----+
|i94port|count|
+-------+-----+
|    PHF|    1|
|    NC8|    1|
|    CNC|    1|
|    VNB|    1|
|    COO|    1|
|    ERC|    1|
|    CPX|    1|
|    PCF|    1|
|    LWT|    1|
|    NIG|    1|
|    MAI|    1|
|    RIO|    1|
|    HNN|    1|
|    YIP|    1|
|    ANA|    1|
|    SCH|    1|
|    BWM|    1|
|    REN|    1|
|    MND|    1|
|    BHX|    2|
+-------+-----+
only showing top 20 rows



* The above descriptive results show that both 'i94yr' and 'i94mon' columns are clean, but there are lots of invalid codes in 'i94addr' and 'i94port' columns.

* We assumed that all immigrants came to the US on the first day of a given month.

In [13]:
# filter out valid ports
validPorts = get_valid_ports()

In [14]:
# clean immigration data: remove null values, invalid ports and cities
clean_immigration_usa_df = clean_immigration_data(validPorts, immigration_usa_df, spark)

In [15]:
# print schema
clean_immigration_usa_df.printSchema()

root
 |-- dt: date (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- i94port: string (nullable = true)



In [16]:
# create a view of clean immigration data
clean_immigration_usa_df.createOrReplaceTempView('clean_immigration_usa_table')

In [17]:
# check if invalid data are cleaned out
value_counts('clean_immigration_usa_table', 'i94addr')
value_counts('clean_immigration_usa_table', 'i94port')

+-------+-----+
|i94addr|count|
+-------+-----+
|     99|   52|
|     VI|  226|
|     WY|  460|
|     SD|  557|
|     WV|  808|
|     ND| 1225|
|     MT| 1339|
|     VT| 1477|
|     AK| 1604|
|     ID| 1752|
|     MS| 1771|
|     NM| 1994|
|     ME| 2361|
|     NH| 2817|
|     AR| 2873|
|     DE| 3111|
|     KS| 3224|
|     OK| 3239|
|     RI| 3289|
|     IA| 3391|
+-------+-----+
only showing top 20 rows

+-------+-----+
|i94port|count|
+-------+-----+
|    FRI|    1|
|    VNB|    1|
|    HNN|    1|
|    NC8|    1|
|    SPO|    1|
|    RIO|    1|
|    CPX|    1|
|    MGM|    1|
|    BWM|    1|
|    NIG|    1|
|    ANA|    1|
|    LWT|    1|
|    PHF|    1|
|    YIP|    1|
|    MND|    1|
|    PSM|    2|
|    ADW|    2|
|    NOO|    2|
|    SGJ|    2|
|    MTH|    2|
+-------+-----+
only showing top 20 rows



## Data Wrangling: Temperature data

* We only focus on data from usa

* As the immigration data is from 2016, we only select temperature data as close to this year as possible.

In [18]:
# select needed columns
temperature_usa_df = filter_temperature_data(temperature_df)

In [19]:
# check if only data for April, 2013 are selected
temperature_usa_df.groupBy('dt').count().show()

+----------+-----+
|        dt|count|
+----------+-----+
|2013-04-01|  257|
+----------+-----+



In [25]:
# add intry ports infos to temperature_usa_df (extracted from immigration data based on a given city)
validPorts = get_valid_ports()
temperature_usa_df = add_port_to_temperature_data(temperature_usa_df, validPorts)

In [26]:
# print schema
temperature_usa_df.printSchema()

root
 |-- dt: date (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- i94port: string (nullable = true)



In [27]:
# visualize temperature data
temperature_usa_df.limit(10).toPandas()

,dt,Country,City,AverageTemperature,i94port
0,2013-04-01,United States,Abilene,15.752999999999998,None
1,2013-04-01,United States,Akron,9.691,AKR
2,2013-04-01,United States,Albuquerque,11.555,ABQ
3,2013-04-01,United States,Alexandria,12.425,AXB
4,2013-04-01,United States,Allentown,9.722999999999999,None
5,2013-04-01,United States,Amarillo,12.954,None
6,2013-04-01,United States,Anaheim,15.380999999999998,None
7,2013-04-01,United States,Anchorage,-6.421,ANC
8,2013-04-01,United States,Ann Arbor,6.819,None
9,2013-04-01,United States,Antioch,15.995999999999999,None


In [28]:
# deacribing Average Temperature data column
temperature_usa_df.select(['AverageTemperature']).describe().show()

+-------+------------------+
|summary|AverageTemperature|
+-------+------------------+
|  count|               257|
|   mean| 13.75025680933851|
| stddev| 5.253205757503348|
|    min|            -0.591|
|    max| 9.722999999999999|
+-------+------------------+



In [29]:
# create temporal view of temperature_usa data
temperature_usa_df.createOrReplaceTempView('temperature_usa_table')

In [30]:
# count unique values in each column
for col in temperature_usa_df.columns: value_counts('temperature_usa_table', col)

+----------+-----+
|        dt|count|
+----------+-----+
|2013-04-01|  257|
+----------+-----+

+-------------+-----+
|      Country|count|
+-------------+-----+
|United States|  257|
+-------------+-----+

+---------------+-----+
|           City|count|
+---------------+-----+
|    Chattanooga|    1|
|      Worcester|    1|
|     Charleston|    1|
|          Tempe|    1|
|         Corona|    1|
|       Thornton|    1|
|North Las Vegas|    1|
|        Phoenix|    1|
|      Hollywood|    1|
|       Savannah|    1|
|     Toms River|    1|
| Pembroke Pines|    1|
|  Coral Springs|    1|
|          Omaha|    1|
|      Anchorage|    1|
|       Paradise|    1|
|      Allentown|    1|
|   Fort Collins|    1|
|        Anaheim|    1|
|     Greensboro|    1|
+---------------+-----+
only showing top 20 rows

+------------------+-----+
|AverageTemperature|count|
+------------------+-----+
|             17.16|    1|
|            21.193|    1|
|             2.543|    1|
|15.752999999999998|    1|
| 

* Adding i94port column to temprature dataframe. It is mapped from cleaned up immigration dataframe.

In [6]:
validPorts = get_valid_ports()
temperature_usa_df = add_port_to_temperature_data(temperature_usa_df, validPorts)

In [7]:
temperature_usa_df.createOrReplaceTempView('temperature_usa_table')

In [10]:
clean_temerature_usa_df = clean_temerature_usa_data(temperature_usa_df, spark)

In [11]:
clean_temerature_usa_df.show(5)

+----------+-------------+-----------+------------------+-------+
|        dt|      Country|       City|AverageTemperature|i94port|
+----------+-------------+-----------+------------------+-------+
|2013-04-01|United States|      Akron|              9.69|    AKR|
|2013-04-01|United States|Albuquerque|             11.56|    ABQ|
|2013-04-01|United States| Alexandria|             12.43|    AXB|
|2013-04-01|United States|  Anchorage|             -6.42|    ANC|
|2013-04-01|United States|    Atlanta|             14.53|    ATL|
+----------+-------------+-----------+------------------+-------+
only showing top 5 rows



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model
* **Data model**:
    * The star schema is used as datamodel of this project.
    * It is a relational model contains one fact table named fact_table surrounded by two dimension tables named dim_immigration_table and dim_temperature_table.
    * It suits analytical queries and user can analyze the data with few number of joins.


* **Fact table** - dim_immigration table joined with the dim_temperature table on i94port and dt, Columns:
    * dt - date of arrival,
    * i94port: 3 character code of destination USA city,
    * AverageTemperature: average temperature of destination city


* **Immigration dimension table**:
    * dt - date of arrival,
    * i194addr: where the immigrants resides in USA (2 character code),
    * i94port: 3 character code of destination USA city


* **Temperature dimension table**:
    * dt: date of recorded temperature
    * AverageTemperature: average temperature
    * City: city name
    * Country: country name
    * i94port: 3 character code of entry port (extracted from i94-immigration data)



#### 3.2 Mapping Out Data Pipelines
* Import all neccassery libraries and functions
* Initialize reader object and assign spark and data to appropriate variables
* Process immigration data to create clean immigration dimention table
    * filter, clean and load data
* Process temperature data to create clean temperature dimention table
    * filter, clean and load data
* Preate fact table from dimention tables
    * join dimention tables and load data
* Perform data quality check
    * Count check to ensure completeness

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [1]:
# do all imports and installs here
import logging
logging.getLogger().setLevel(logging.INFO)
import pyspark.sql.dataframe as psd
import pyspark.sql.session as pss
from pyspark.sql import SparkSession
from utils.reader import Reader
from utils.help_functions import *

In [2]:
# initialize reader classs
reader = Reader()

# assign spark, immigration data, and temperature data to corresponding valiables
spark = reader.spark
immigration_df = reader.immigration_df
temperature_df = reader.temperature_df

INFO:root:geting root paths to read and write data
INFO:root:getting spark session
INFO:root:stage raw data into parquet files if not already exist!!!
INFO:root:i94 immigration raw data are are already written into parquet files
INFO:root:world temperature raw data are are already written into parquet files
INFO:root:Start fitting i94immigration data to reader class ...
INFO:root:End fitting i94immigration data to reader class!
INFO:root:Start fitting worldtemperature data to reader class ...
INFO:root:End fitting worldtemperature data to reader class!


In [3]:
def process_immigration_data(immigration_df: psd.DataFrame, spark: pss.SparkSession):
    # extract columns to create immigration_usa table
    immigration_usa_df = filter_immigration_data(immigration_df)

    # create temporal view of immigration_usa data
    immigration_usa_df.createOrReplaceTempView('immigration_usa_table')

    # remove null values from immigration_usa columns
    validPorts = get_valid_ports()
    clean_immigration_usa_df = clean_immigration_data(validPorts, immigration_usa_df, spark)

    # write clean immigration_usa table to parquet files
    print("Start writting immigration_usa table to output path ....")
    clean_immigration_usa_df.write.mode("overwrite").partitionBy("i94port").parquet("output_data/immigration.parquet")
    print("End writting immigration_usa table to output path!")
    
    # create a view of clean immigration_usa data for further analysis
    clean_immigration_usa_df.createOrReplaceTempView('clean_immigration_usa_table')

In [4]:
def process_temperature_data(temperature_df: psd.DataFrame, spark: pss.SparkSession):
    # extract columns to create temperature_usa table
    temperature_usa_df = filter_temperature_data(temperature_df)
    
    # add intry ports infos to temperature_usa_df
    validPorts = get_valid_ports()
    temperature_usa_df = add_port_to_temperature_data(temperature_usa_df, validPorts)
    
    # create temporal view of temperature_usa data
    temperature_usa_df.createOrReplaceTempView('temperature_usa_table')

    # remove null values from temperature_usa columns
    clean_temperature_usa_df = clean_temerature_usa_data(temperature_usa_df, spark)

    # write clean temperature_usa table to parquet files
    print("Start writting temperature_usa table to output path ....")
    clean_temperature_usa_df.write.mode("overwrite").partitionBy("i94port").parquet("output_data/temprature.parquet")
    print("End writting temperature_usa table to output path!")
    
    # create a view of clean temperature_usa data for further analysis
    clean_temperature_usa_df.createOrReplaceTempView('clean_temperature_usa_table')

In [5]:
def process_fact_table(spark: pss.SparkSession):
    # extract columns to create fact table by joining clean_temperature_usa_table and clean_immigration_usa_table
    fact_df = spark.sql(query_create_fact_table())
    
    # write clean fact table to parquet files
    print("Start writting fact table to output path ....")
    fact_df.write.mode("overwrite").partitionBy("i94port").parquet("output_data/fact.parquet")
    print("End writting fact table to output path!")
    
    # create a view of fact data for further analysis
    fact_df.createOrReplaceTempView('fact_table')

In [6]:
def immigration_movement_etl():
    # proccess immigration data to create immigration dimention table
    process_immigration_data(immigration_df, spark)
    
    # proccess world tempetature data to create temperature dimention table
    process_temperature_data(temperature_df, spark)
    
    # proccess created dimention tables to create fact table
    process_fact_table(spark)

In [7]:
# run ETL pipeline 
immigration_movement_etl()

Start writting immigration_usa table to output path ....
End writting immigration_usa table to output path!
Start writting temperature_usa table to output path ....
End writting temperature_usa table to output path!
Start writting fact table to output path ....
End writting fact table to output path!


#### 4.2 Data Quality Checks
* Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
    * Check the number of netries to ensure completeness
  
Run Quality Checks

In [8]:
# Perform quality checks
check_exist_rows(['clean_immigration_usa_table', 'clean_temperature_usa_table', 'fact_table'], spark)

INFO:root:Running data quality check on table clean_immigration_usa_table
INFO:root:Getting number of entries in table clean_immigration_usa_table
INFO:root:Table clean_immigration_usa_table has [2917199] numbers of entries.
INFO:root:Data quality check passed for table clean_immigration_usa_table
INFO:root:Running data quality check on table clean_temperature_usa_table
INFO:root:Getting number of entries in table clean_temperature_usa_table
INFO:root:Table clean_temperature_usa_table has [117] numbers of entries.
INFO:root:Data quality check passed for table clean_temperature_usa_table
INFO:root:Running data quality check on table fact_table
INFO:root:Getting number of entries in table fact_table
INFO:root:Table fact_table has [2917199] numbers of entries.
INFO:root:Data quality check passed for table fact_table


In [9]:
# check if the number of entries in fact table is same as clean_immigration_usa_table
check_tables_integrity('fact_table', 'clean_immigration_usa_table', spark)

INFO:root:Running data integrity check on fact and dimmention tables
INFO:root:Getting number of entries in table clean_immigration_usa_table
INFO:root:Table clean_immigration_usa_table has [2917199] numbers of entries.
INFO:root:Getting number of entries in table fact_table
INFO:root:Table fact_table has [2917199] numbers of entries.
INFO:root:Data integrity check passed for tables fact_table and clean_immigration_usa_table


In [10]:
# check if null values are not existed in fact table for dt and i94port columns
check_constrain_integrity('fact_table', spark)

INFO:root:Running data constrain check on fact table
INFO:root:Getting number of null values in table fact_table
INFO:root:Constrain integrity check passed for tables fact_table


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

Included in **utils/DataDictionary.md**

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

* In this project I used Apache Spark to do all of the ETL data processing. It is capable of scaling a large amount of data sets and has very useful functions for cleaning up and wangling data. In this project, we only examined data recorded over a month. We can easily expand the project scope and model several months of data because the use of the parquet format can easily be scaled to a much larger dataset. If we choose to examine data on a monthly basis, Apache Airflow can be used to partition data by month and analyze them separately.

* I would approach the problem as follows under different conditions:

    * If the data has increased by 100x: Spark can still process it, one may need to use more cluster nodes. I would also consider using the Redshift Analytical database as it is optimized for aggregation and performs very well on heavy workloads. Based on the size of our dataset, we can adjust the size of the EMR cluster.

    * If my job was to update data on a daily basis, I would definitely use Apache Airflow to create a schedule for the update.

    * If more than 100 people need to access the data, more CPU resources are needed. By using a distributed database like Redshift database we can improve our replications and partitioning, so that users can get faster query results.